# 部署到批处理终结点

假设一个健康诊所收集患者的全天测量值，将每位患者的详细信息保存在单独的文件中。 然后在夜间，糖尿病预测模型可用于批量处理一整天的患者数据，生成第二天早上需要的预测，这样诊所就可以跟踪预测有患糖尿病风险的患者。 借助 Azure 机器学习，可以通过创建批处理终结点来做到这一点，你将在本练习中完成此操作。

## 准备工作

需要最新版本的 azureml-ai-ml 包才能运行此笔记本中的代码。 运行下面的单元以验证是否已安装它。

> **注意**：
> 如果未安装“azure-ai-ml”包，请运行 `pip install azure-ai-ml` 以进行安装。

In [ ]:
## 连接到工作区

安装必需的 SDK 包后，就可以连接到工作区了。

若要连接到工作区，我们需要标识符参数 - 订阅 ID、资源组名称和工作区名称。 已为你填写资源组名称和工作区名称。 只需订阅 ID 即可完成命令。

若要查找所需的参数，请单击工作室右上角的订阅和工作区名称。 右侧将打开一个窗格。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 复制订阅 ID，并将“YOUR-SUBSCRIPTION-ID”替换为复制的值。 </p>

## 注册模型

批处理部署只能部署已在工作区中注册的模型。 你将注册存储在本地 `model` 文件夹中的 MLflow 模型。 

In [ ]:
## 创建批处理终结点

批处理终结点是应用程序可以调用以触发批量评分作业的 HTTPS 终结点。 批处理终结点名称需要在 Azure 区域中是唯一的。 你将使用 `datetime` 函数根据当前日期和时间生成唯一名称。 

In [ ]:
若要使用 `BatchEndpoint` 类创建终结点，需要指定名称和说明（可选）。 创建终结点后，要将模型部署到终结点。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 重要说明！ 请等待终结点创建，然后再继续操作！ 工作室中应显示绿色通知。 </p>

## 创建部署

部署是一组资源，用于承载执行实际推理的模型。 我们将使用 `BatchDeployment` 类为终结点创建一个部署。 

由于要部署 MLflow 模型，因此不需要评分脚本或定义环境。 Azure 机器学习会自动为你创建这些资产。 `model` 文件夹中的 `MLmodel` 文件用于了解模型的预期输入和输出。

你将使用以下参数部署模型：

- `name`：部署的名称。
- `description`：用于进一步阐明部署表示的内容的可选说明。
- `endpoint_name`：模型应部署到的以前创建的终结点的名称。
- `model`：已注册模型的名称。
- `compute`：调用部署的模型以生成预测时使用的计算。
- `instance_count`：用于生成预测的计算节点计数。
- `max_concurrency_per_instance`：每个计算节点运行的最大并行评分脚本数。
- `mini_batch_size`：每个评分脚本运行传递的文件数。
- `output_action`：每个新预测都将作为新行追加到输出文件。
- `output_file_name`：预测将追加到的文件。
- `retry_settings`：针对微型批处理失败的设置。
- `logging_level`：日志详细程度。 允许使用的值为 `warning`、`info` 和 `debug`。 

运行以下单元格将配置并创建部署。

In [ ]:
<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 重要说明！ 请等待部署完成，然后再继续操作！ 工作室中应显示绿色通知。 </p>

可以将多个模型部署到一个终结点。 可以设置默认部署，以指定调用批处理终结点时应默认使用哪个模型。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 重要说明！ 请等待默认部署设置，然后再继续操作！ 工作室中应显示绿色通知。 </p>

## 准备用于批量预测的数据

在 `data` 文件夹中，你将找到包含未标记数据的 CSV 文件。 你将创建指向 `data` 文件夹中的文件的数据资产，这将用作批处理作业的输入。

In [ ]:
## 提交作业

将模型部署到批处理终结点并准备好未标记的数据资产后，即可调用终结点以针对未标记的数据生成预测。

首先通过引用已注册的数据资产来定义输入。 然后调用终结点，该终结点将提交管道作业。 可以使用作业 URL 在工作室中对其进行监视。 该作业将包含一个子作业，表示用于获取预测信息的（生成的）评分脚本的运行。

## 获取结果

完成调用批处理终结点的管道作业后，可以查看结果。 所有预测都收集在默认数据存储中存储的 `predictions.csv` 文件中。 可以通过运行以下单元格来下载文件并将数据可视化。 

In [ ]:
from azure.ai.ml.entities import BatchEndpoint

# create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for classifying diabetes in patients",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the endpoint is created before continuing! A green notification should appear in the studio. </p>

## Create the deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `BatchDeployment` class. 

Since you're deploying an MLflow model, you don't need a scoring script or define the environment. Azure Machine Learning will automatically create those assets for you. The `MLmodel` file in the `model` folder is used to understand what the expected inputs and outputs are of the model.

You'll deploy a model with the following parameters:

- `name`: Name of the deployment.
- `description`: Optional description to further clarify what the deployment represents.
- `endpoint_name`: Name of the previously created endpoint the model should be deployed to.
- `model`: Name of the registered model.
- `compute`: Compute to be used when invoking the deployed model to generate predictions.
- `instance_count`: Count of compute nodes to use for generating predictions.
- `max_concurrency_per_instance`: Maximum number of parallel scoring script runs per compute node.
- `mini_batch_size`: Number of files passed per scoring script run.
- `output_action`: Each new prediction will be appended as a new row to the output file.
- `output_file_name`: File to which predictions will be appended.
- `retry_settings`: Settings for a mini-batch fails.
- `logging_level`: The log verbosity level. Allowed values are `warning`, `info`, and `debug`. 

Running the following cell will configure and create the deployment.

In [ ]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

deployment = BatchDeployment(
    name="classifier-diabetes-mlflow",
    description="A diabetes classifier",
    endpoint_name=endpoint.name,
    model=model,
    compute="aml-cluster",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)
ml_client.batch_deployments.begin_create_or_update(deployment)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio. </p>

You can deploy multiple models to an endpoint. You can set the default deployment to specify which model should be used by default when calling a batch endpoint.

In [ ]:
endpoint.defaults = {}

endpoint.defaults["deployment_name"] = deployment.name

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the default deployment is set before continuing! A green notification should appear in the studio. </p>

## Prepare the data for batch predictions

In the `data` folder you'll find CSV files with unlabeled data. You'll create a data asset that points to the files in the `data` folder, which you'll use as input for the batch job.

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "./data"
dataset_name = "patient-data-unlabeled"

patient_dataset_unlabeled = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="An unlabeled dataset for diabetes classification",
    name=dataset_name,
)
ml_client.data.create_or_update(patient_dataset_unlabeled)

In [ ]:
patient_dataset_unlabeled = ml_client.data.get(
    name="patient-data-unlabeled", label="latest"
)

## Submit the job

Now that you have deployed a model to a batch endpoint, and have an unlabeled data asset, you're ready to invoke the endpoint to generate predictions on the unlabeled data.

First, you'll define the input by referring to the registered data asset. Then, you'll invoke the endpoint, which will submit a pipeline job. You can use the job URL to monitor it in the Studio. The job will contain a child job that represents the running of the (generated) scoring script to get the predictions.

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

input = Input(type=AssetTypes.URI_FOLDER, path=patient_dataset_unlabeled.id)

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
    input=input)

ml_client.jobs.get(job.name)

## Get the results

When the pipeline job that invokes the batch endpoint is completed, you can view the results. All predictions are collected in the `predictions.csv` file that is stored in the default datastore. You can download the file and visualize the data by running the following cells. 

In [ ]:
ml_client.jobs.download(name=job.name, download_path=".", output_name="score")

In [ ]:
with open("predictions.csv", "r") as f:
    data = f.read()

In [ ]:
from ast import literal_eval
import pandas as pd

score = pd.DataFrame(
    literal_eval(data.replace("\n", ",")), columns=["file", "prediction"]
)
score